In [ ]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import matplotlib
from datetime import datetime

In [ ]:
class MaCrossStrategy(bt.Strategy):
 
    params = (
        ('fast_length', 5),
        ('slow_length', 25), 
        ('vix_level', 30)
    )
     
    def __init__(self):
        ma_fast = bt.ind.SMA(period = self.params.fast_length)
        ma_slow = bt.ind.SMA(period = self.params.slow_length)
         
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)
        self.vix = self.data1.close
 
    def next(self):
        if not self.position:
            if self.crossover > 0 and self.vix < self.params.vix_level: 
                self.buy()
        elif self.crossover < 0: 
            self.close()

In [ ]:
cerebro = bt.Cerebro()
 
data = bt.feeds.YahooFinanceData(dataname = 'AAPL', fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
cerebro.adddata(data)
 
vix = bt.feeds.YahooFinanceData(dataname = '^VIX', fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
cerebro.adddata(vix)
 
cerebro.addstrategy(MaCrossStrategy, fast_length = 5, slow_length = 25, vix_level = 999)
cerebro.broker.setcash(1000000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 50)
 
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Returns,     _name = "returns")
 
back = cerebro.run()
 
cerebro.broker.getvalue()

In [ ]:
back[0].analyzers.sharpe.get_analysis()['sharperatio']

In [ ]:
back[0].analyzers.returns.get_analysis()['rnorm100']